# Biodata Mahasiswa
## Praktikum Pekan 10 : Regresi menggunakan Model Neural Network (NN) pada IMDB Dataset

VSI2J3 - Dasar Ilmu Data

NIM: 607012300120

Nama: Ilham Setiawan Siregar

Kelas: D3SI-47-04

# Deskripsi

Colab berikut merupakan contoh untuk memprediksi sentimen (positif/Mengelompokkan rating numerik (misalnya 1–10) menjadi kategori ordinal (contoh: 1–3 = Buruk, 4–7 = Sedang, 8–10 = Baik).
Kolom Target: ratings yang diubah menjadi label kategorikal.

# Tahapan code untuk persiapan data
1. Koneksi colab dengan GDrive
3. Import library yang dibutuhkan
4. Load dataset
5. Tampilkan contoh data

## Koneksi colab dengan GDrive





In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Download data NLTK untuk preprocessing teks
nltk.download('stopwords')
nltk.download('punkt')

# Set seed untuk reproduktibilitas
np.random.seed(42)
tf.random.set_seed(42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load dataset

In [ ]:
#load data google drive
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/imdb_sup.csv')
print (df.shape)

(50000, 3)


## Tampilkan contoh data

In [ ]:
df.head(100)

,Review,Rating,Sentiment
0,"Kurt Russell's chameleon-like performance, cou...",10,1
1,It was extremely low budget(it some scenes it ...,8,1
2,James Cagney is best known for his tough chara...,8,1
3,"Following the brilliant ""Goyôkiba"" (aka. ""Hanz...",8,1
4,One of the last classics of the French New Wav...,10,1
...,...,...,...
95,I really liked this version of 'Vanishing Poin...,8,1
96,Colman's performance is aided by the brilliant...,10,1
97,Michelle Rodriguez is a well-built high-school...,7,1
98,Henry Fonda brilliantly captures what we have ...,9,1


# B. Regresi menggunakan Model NN

## 2) Pilih fitur yang akan digunakan

In [ ]:
# Code pemilihan fitur dan label
X_text = df['Review']  # Fitur utama: teks ulasan
X_sentiment = df['Sentiment']  # Fitur tambahan: sentimen biner (opsional)
y = df['Rating']  # Label: rating numerik (akan diubah menjadi kategorikal)

# Jika ingin menggunakan Sentiment sebagai fitur tambahan, gabungkan dengan teks nanti
# Untuk saat ini, hanya gunakan Review sebagai fitur utama
X = X_text

## 3) Lakukan pre-processing (jika diperlukan)

In [ ]:
# kode pre-processing
# Preprocessing teks dan label
def preprocess_text(text):
    text = text.lower()  # Konversi ke lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Hapus karakter khusus dan angka
    tokens = word_tokenize(text)  # Tokenisasi
    stop_words = set(stopwords.words('english'))  # Stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Hapus stopwords
    return ' '.join(tokens)

# Terapkan preprocessing pada kolom Review
X_text = X_text.apply(preprocess_text)

# Konversi rating menjadi kategori ordinal
def categorize_rating(rating):
    if rating <= 3:
        return 'Buruk'
    elif rating <= 7:
        return 'Sedang'
    else:
        return 'Baik'

y = y.apply(categorize_rating)

# Encode label kategorikal menjadi numerik
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot encoding untuk klasifikasi

# Tokenisasi teks
max_words = 5000  # Jumlah kata maksimum untuk kamus
max_len = 200     # Panjang maksimum sequence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_text)
X_sequences = tokenizer.texts_to_sequences(X_text)
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Opsional: Gabungkan Sentiment sebagai fitur tambahan
# X_combined = np.hstack((X_padded, X_sentiment.values.reshape(-1, 1)))
# Untuk saat ini, gunakan hanya X_padded
X = X_padded

## 4) Pembagian Dataset Training dan Testing

In [ ]:
# Pembagian dataset menjadi training dan testing
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y_categorical, df.index, test_size=0.2, random_state=42
)

print("Ukuran data training:", X_train.shape)
print("Ukuran data testing:", X_test.shape)

Ukuran data training: (40000, 200)
Ukuran data testing: (10000, 200)


## 5) Siapkan regressor, tentukan variabel dan parameter

In [ ]:
# Code regressor
# Siapkan model Neural Network
embedding_dim = 100  # Dimensi embedding

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    SpatialDropout1D(0.2),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 kelas: Buruk, Sedang, Baik
])

# Kompilasi model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Tampilkan ringkasan model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 6) Training regressor pada dataset


In [ ]:
# Code training regressor pada dataset

# Training model
epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 222s 347ms/step - accuracy: 0.5881 - loss: 0.8830 - val_accuracy: 0.7246 - val_loss: 0.6551
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 214s 343ms/step - accuracy: 0.7468 - loss: 0.6221 - val_accuracy: 0.7288 - val_loss: 0.6516
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 339ms/step - accuracy: 0.7742 - loss: 0.5626 - val_accuracy: 0.7138 - val_loss: 0.6776
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 340ms/step - accuracy: 0.7918 - loss: 0.5214 - val_accuracy: 0.7129 - val_loss: 0.7516
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 264s 343ms/step - accuracy: 0.8121 - loss: 0.4774 - val_accuracy: 0.7062 - val_loss: 0.7688
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 345ms/step - accuracy: 0.8284 - loss: 0.4413 - val_accuracy: 0.6989 - val_loss: 0.8286
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 345ms/step - accuracy: 0.8445 - loss: 0.4060 - val_accuracy: 0.6911 - val_loss: 0.8939
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 345ms/step - accuracy: 0.8546 -

## 7) Lakukan pengujian dengan data uji (melakukan prediksi)



In [ ]:
# Code pengujian (melakukan prediksi)

# Prediksi pada data uji
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Konversi probabilitas ke kelas
y_test_classes = np.argmax(y_test, axis=1)  # Konversi one-hot ke kelas


313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 53ms/step


In [ ]:
# Tampilkan hasil prediksi
print("Contoh Hasil Prediksi (10 data pertama):")
for i, idx in enumerate(test_indices[:10]):
    print(f"Review {i+1}:")
    print(f"Teks: {df['Review'].iloc[idx][:100]}...")
    print(f"Sentimen (Biner): {df['Sentiment'].iloc[idx]}")
    print(f"Prediksi: {label_encoder.inverse_transform([y_pred_classes[i]])[0]}")
    print(f"Sebenarnya: {label_encoder.inverse_transform([y_test_classes[i]])[0]}")
    print()

# Tampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

Contoh Hasil Prediksi (10 data pertama):
Review 1:
Teks: Having read all of the comments on this film I am still amazed at Fox's reluctance to release a full...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 2:
Teks: I like this film a lot. It has a wonderful chemistry between the actors and tells a story that is pr...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 3:
Teks: I am a huge fan of Simon Pegg and have watched plenty of his movies until now and none of them have ...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Sedang

Review 4:
Teks: This was what black society was like before the crack epidemics, gangsta rap, and AIDS that beset th...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 5:
Teks: pretty disappointing. i was expecting more of a horror/thriller -- but this seemed to be more of an ...
Sentimen (Biner): 0
Prediksi: Sedang
Sebenarnya: Buruk

Review 6:
Teks: As a flagship show, Attack of the Show (AOTS) is endemic of the larger fa

## 8) Analisa performansi model

### a) Menggunakan Mean Absolute Error (MAE)

In [ ]:
# Code analisa performansi menggunakan MAE

from sklearn.metrics import mean_absolute_error

# Hitung MAE
mae = mean_absolute_error(y_test_classes, y_pred_classes)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Mean Absolute Error (MAE): 0.4410


### b) Menggunakan Mean Squared Error (MSE)

In [ ]:
# Code analisa performansi menggunakan MSE

from sklearn.metrics import mean_squared_error

# Hitung MSE
mse = mean_squared_error(y_test_classes, y_pred_classes)
print(f"Mean Squared Error (MSE): {mse:.4f}")

Mean Squared Error (MSE): 0.6966


### c) Menggunakan RMSE

In [ ]:
# Code analisa performansi menggunakan RMSE

from sklearn.metrics import mean_squared_error
import numpy as np

# Hitung RMSE
rmse = np.sqrt(mean_squared_error(y_test_classes, y_pred_classes))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Root Mean Squared Error (RMSE): 0.8346


### d) Menggunakan R squared

In [ ]:
# Code analisa performansi menggunakan R squared

from sklearn.metrics import r2_score

# Hitung R-squared
r2 = r2_score(y_test_classes, y_pred_classes)
print(f"R-squared (R²): {r2:.4f}")

R-squared (R²): -0.2159


## 9) Kesimpulan sebelum HPO

 1. Model Klasifikasi: Cukup Baik, Tapi Masih Perlu Peningkatan
Akurasi model : 69%, dengan F1-score rata-rata 0.68.
Performa terbaik pada kelas "Baik" dan "Buruk" , tetapi sangat lemah pada kelas "Sedang" (F1-score hanya 0.36).
Model lebih cenderung memprediksi kelas mayoritas ("Baik" atau "Buruk"), menunjukkan kemungkinan adanya ketidakseimbangan data atau kurangnya pola jelas untuk kelas "Sedang".

 2. Model Regresi: Tidak Berhasil
R-squared negatif (-0.2159) → model gagal menjelaskan variasi target, bahkan lebih buruk dari rata-rata baseline.
RMSE = 0.8346 dan MAE = 0.4410 → error cukup besar, terutama jika rentang target kecil (misalnya 0–1 atau 1–5).

 3. Overfitting Jelas Terjadi Selama Pelatihan
Akurasi training meningkat pesat hingga 88% , tapi validasi turun dari 72% ke 68% .
Loss validasi meningkat drastis dari ~0.65 ke ~1.09, menandakan model gagal generalisasi.

Kesimpulan Utama:
Model klasifikasi saat ini masih layak dikembangkan , namun membutuhkan optimisasi lebih lanjut untuk meningkatkan performa, terutama pada kelas "Sedang".
Model regresi tidak berhasil dan sebaiknya ditinjau ulang atau diganti dengan pendekatan klasifikasi saja.
Dengan optimisasi hyperparameter dan penyempurnaan arsitektur/model, hasil dapat ditingkatkan secara signifikan.



## 10) Optimasi: HPO dengan RandomizedSearchCV

In [ ]:
# kode Optimasi: HPO dengan RandomizedSearchCV

!pip install keras-tuner
import keras_tuner as kt
import tensorflow as tf # Pastikan tensorflow diimpor karena digunakan di dalam build_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout

# Definisikan fungsi untuk membangun model dengan hyperparameter
def build_model(hp):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('lstm_units_1', min_value=32, max_value=128, step=32), return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units_2', min_value=16, max_value=64, step=16)))
    model.add(Dense(units=hp.Int('dense_units', min_value=16, max_value=64, step=16), activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(3, activation='softmax'))

    # Kompilasi model dengan learning rate yang dioptimasi
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Inisialisasi tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Jumlah kombinasi hyperparameter yang akan dicoba
    executions_per_trial=1,  # Jumlah eksekusi per kombinasi
    directory='hpo_dir',
    project_name='imdb_sentiment'
)

# Jalankan pencarian hyperparameter
# Pastikan X_train, y_train, X_test, y_test, max_words, embedding_dim, dan max_len sudah didefinisikan
# dari sel-sel sebelumnya.
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test), verbose=1)

# Dapatkan model terbaik
best_model = tuner.get_best_models(num_models=1)[0]
# Hapus num_models=1 dari get_best_hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

# Tampilkan hyperparameter terbaik
print("Hyperparameter Terbaik:")
print(f"- LSTM Units 1: {best_hyperparameters.get('lstm_units_1')}")
print(f"- LSTM Units 2: {best_hyperparameters.get('lstm_units_2')}")
print(f"- Dense Units: {best_hyperparameters.get('dense_units')}")
print(f"- Spatial Dropout: {best_hyperparameters.get('spatial_dropout'):.2f}")
print(f"- Dropout: {best_hyperparameters.get('dropout'):.2f}")
print(f"- Learning Rate: {best_hyperparameters.get('learning_rate'):.6f}")

# Ringkasan model terbaik
best_model.summary()

Reloading Tuner from hpo_dir/imdb_sentiment/tuner0.json
Hyperparameter Terbaik:
- LSTM Units 1: 64
- LSTM Units 2: 64
- Dense Units: 16
- Spatial Dropout: 0.40
- Dropout: 0.40
- Learning Rate: 0.000185


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 200, 100)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (32, 200, 100)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 200, 64)          │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (32, 64)               │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 16)               │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 16)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 3)                │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 576,355 (2.20 MB)

 Trainable params: 576,355 (2.20 MB)

 Non-trainable params: 0 (0.00 B)

## 11) Implementasi best parameter pada model

Siapkan regressor ke 2 dan tentukan variabel/parameternya

In [ ]:
# Code untuk regressor ke 2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout

# Ambil hyperparameter terbaik dari HPO
lstm_units_1 = best_hyperparameters.get('lstm_units_1')
lstm_units_2 = best_hyperparameters.get('lstm_units_2')
dense_units = best_hyperparameters.get('dense_units')
spatial_dropout = best_hyperparameters.get('spatial_dropout')
dropout = best_hyperparameters.get('dropout')
learning_rate = best_hyperparameters.get('learning_rate')

# Siapkan regressor ke-2 dengan hyperparameter terbaik
model_2 = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    SpatialDropout1D(spatial_dropout),
    LSTM(lstm_units_1, return_sequences=True),
    LSTM(lstm_units_2),
    Dense(dense_units, activation='relu'),
    Dropout(dropout),
    Dense(3, activation='softmax')  # 3 kelas: Buruk, Sedang, Baik
])

# Kompilasi model
model_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Tampilkan ringkasan model
print("Ringkasan Regressor Ke-2:")
model_2.summary()


Ringkasan Regressor Ke-2:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 12) Lakukan proses training dengan data latih

In [ ]:
# Code training pada regressor ke 2

# Training regressor ke-2
epochs = 10  # Menggunakan jumlah epoch yang sama dengan model awal untuk konsistensi
batch_size = 64  # Menggunakan batch size yang sama dengan model awal

history_2 = model_2.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 254s 394ms/step - accuracy: 0.4936 - loss: 0.9943 - val_accuracy: 0.7068 - val_loss: 0.6847
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 261s 393ms/step - accuracy: 0.7178 - loss: 0.6896 - val_accuracy: 0.7287 - val_loss: 0.6493
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 259s 387ms/step - accuracy: 0.7460 - loss: 0.6291 - val_accuracy: 0.7352 - val_loss: 0.6360
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 245s 392ms/step - accuracy: 0.7582 - loss: 0.5941 - val_accuracy: 0.7272 - val_loss: 0.6473
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 392ms/step - accuracy: 0.7708 - loss: 0.5724 - val_accuracy: 0.7264 - val_loss: 0.6580
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 394ms/step - accuracy: 0.7747 - loss: 0.5572 - val_accuracy: 0.7310 - val_loss: 0.6628
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 264s 398ms/step - accuracy: 0.7838 - loss: 0.5389 - val_accuracy: 0.7217 - val_loss: 0.6744
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 395ms/step - accuracy: 0.7909 -

## 13) Lakukan pengujian dengan data uji

In [ ]:
# Code pengujian pada data test menggunakan regressor ke 2
# Prediksi pada data uji menggunakan regressor ke-2
y_pred_2 = model_2.predict(X_test)
y_pred_classes_2 = np.argmax(y_pred_2, axis=1)  # Konversi probabilitas ke kelas
y_test_classes = np.argmax(y_test, axis=1)  # Konversi one-hot ke kelas (sudah ada dari sebelumnya)

# Tampilkan contoh hasil prediksi (10 data pertama)
print("Contoh Hasil Prediksi Regressor Ke-2 (10 data pertama):")
for i, idx in enumerate(test_indices[:10]):
    print(f"Review {i+1}:")
    print(f"Teks: {df['Review'].iloc[idx][:100]}...")
    print(f"Sentimen (Biner): {df['Sentiment'].iloc[idx]}")
    print(f"Prediksi: {label_encoder.inverse_transform([y_pred_classes_2[i]])[0]}")
    print(f"Sebenarnya: {label_encoder.inverse_transform([y_test_classes[i]])[0]}")
    print()

# Tampilkan laporan klasifikasi
print("Laporan Klasifikasi Regressor Ke-2:")
print(classification_report(y_test_classes, y_pred_classes_2, target_names=label_encoder.classes_))

313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step
Contoh Hasil Prediksi Regressor Ke-2 (10 data pertama):
Review 1:
Teks: Having read all of the comments on this film I am still amazed at Fox's reluctance to release a full...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 2:
Teks: I like this film a lot. It has a wonderful chemistry between the actors and tells a story that is pr...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 3:
Teks: I am a huge fan of Simon Pegg and have watched plenty of his movies until now and none of them have ...
Sentimen (Biner): 1
Prediksi: Buruk
Sebenarnya: Sedang

Review 4:
Teks: This was what black society was like before the crack epidemics, gangsta rap, and AIDS that beset th...
Sentimen (Biner): 1
Prediksi: Baik
Sebenarnya: Baik

Review 5:
Teks: pretty disappointing. i was expecting more of a horror/thriller -- but this seemed to be more of an ...
Sentimen (Biner): 0
Prediksi: Sedang
Sebenarnya: Buruk

Review 6:
Teks: As a flagship 

## 14) Analisa performansi model

### a) Menggunakan Mean Absolute Error (MAE)

In [ ]:
# Code analisa performansi menggunakan MAE
from sklearn.metrics import mean_absolute_error

# Hitung MAE untuk regressor ke-2
mae_2 = mean_absolute_error(y_test_classes, y_pred_classes_2)
print(f"Mean Absolute Error (MAE) Regressor Ke-2: {mae_2:.4f}")

Mean Absolute Error (MAE) Regressor Ke-2: 0.3900


### b) Menggunakan Mean Squared Error (MSE)

In [ ]:
# Code analisa performansi menggunakan MSE
from sklearn.metrics import mean_squared_error

# Hitung MSE untuk regressor ke-2
mse_2 = mean_squared_error(y_test_classes, y_pred_classes_2)
print(f"Mean Squared Error (MSE) Regressor Ke-2: {mse_2:.4f}")

Mean Squared Error (MSE) Regressor Ke-2: 0.6152


### c) Menggunakan RMSE

In [ ]:
# Code analisa performansi menggunakan RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

# Hitung RMSE untuk regressor ke-2
rmse_2 = np.sqrt(mean_squared_error(y_test_classes, y_pred_classes_2))
print(f"Root Mean Squared Error (RMSE) Regressor Ke-2: {rmse_2:.4f}")

Root Mean Squared Error (RMSE) Regressor Ke-2: 0.7843


### d) Menggunakan R squared

In [ ]:
# Code analisa performansi menggunakan R squared
from sklearn.metrics import r2_score

# Hitung R-squared untuk regressor ke-2
r2_2 = r2_score(y_test_classes, y_pred_classes_2)
print(f"R-squared (R²) Regressor Ke-2: {r2_2:.4f}")

R-squared (R²) Regressor Ke-2: -0.0738


## 15) Kesimpulan setelah HPO

##  1. **Peningkatan Kinerja Model Klasifikasi Setelah Optimisasi**

###  Perbandingan Sebelum dan Sesudah Optimisasi:

| Metrik | Sebelum Optimisasi | Setelah Optimisasi |
|-------|----------------------|---------------------|
| Accuracy | 69% | **72%** |
| F1-score Baik | 0.78 | 0.80 |
| F1-score Buruk | 0.75 | 0.79 |
| F1-score Sedang | 0.36 | **0.40** |

###  Analisis:
- Ada **peningkatan signifikan dalam akurasi** (dari 69% ke 72%) setelah dilakukan **HPO**.
- Performa model pada **kelas "Sedang" juga meningkat**, meskipun masih menjadi titik lemah.
- Precision dan recall untuk **kelas "Baik" dan "Buruk" sangat baik**, menunjukkan bahwa model lebih stabil dalam mengenali sentimen ekstrem.

>  **Kesimpulan**: Model berhasil ditingkatkan performanya melalui tuning hyperparameter, terutama dalam hal generalisasi dan prediksi multikelas.

---

##  2. **Regresi Masih Tidak Efektif Meskipun Lebih Baik**

###  Hasil Regresi Setelah Optimisasi:

- **R-squared (R²)**: -0.0738 → Masih negatif
- **RMSE**: 0.7843
- **MAE**: 0.3900

###  Analisis:
- Walaupun **sedikit membaik dibanding sebelumnya**, nilai R² tetap negatif → model gagal menjelaskan variasi target.
- Error seperti RMSE dan MAE **masih cukup besar relatif terhadap skor target**.
- Prediksi numerik tidak relevan atau sulit diinterpretasi jika tujuan utamanya adalah analisis sentimen kategorikal.




###  Analisis:
- Arsitektur model **cukup sederhana namun efektif** setelah tuning.
- Penggunaan dropout tinggi (0.4) membantu mengurangi overfitting.
- Learning rate kecil (log scale) memberikan konvergensi yang lebih stabil.

>  **Kesimpulan**: Hyperparameter hasil HPO memberikan kontribusi positif dalam peningkatan performa dan pengurangan overfitting.



##  3. **Proses Pelatihan Menunjukkan Stabilitas yang Lebih Baik**

###  Ringkasan Pelatihan:
- Akurasi training naik secara bertahap dari 49% ke 80%
- Validasi mencapai puncak 73.5%, meskipun ada sedikit penurunan di akhir epoch

###  Analisis:
- Dibandingkan sebelumnya, **overfitting berkurang** berkat penggunaan dropout dan learning rate yang dioptimalkan.
- Model **tidak lagi terlalu cepat overfit** dan memiliki kemampuan generalisasi yang lebih baik.

>  **Kesimpulan**: Proses pelatihan lebih stabil setelah HPO, menunjukkan bahwa model siap digunakan untuk inferensi atau deployment.

# Referensi link

A. Scikit-learn library
1. [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

2. [sklearn.neighbors.KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

B. Others
1. [Data Pre-Processing wit Sklearn using Standard and Minmax scaler](https://www.geeksforgeeks.org/data-pre-processing-wit-sklearn-using-standard-and-minmax-scaler/)
2. [Scaling Data Range using Min Max Scaler](https://botbark.com/2019/12/28/scaling-data-range-using-min-max-scaler/)
3. [Train-Test Split for Evaluating Machine Learning Algorithms](https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/)
4. [Accuracy vs. F1-Score](https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2)
5. [Mengenal Accuracy, Precision, Recall dan Specificity serta yang diprioritaskan dalam Machine Learning ](https://rey1024.medium.com/mengenal-accuracy-precission-recall-dan-specificity-serta-yang-diprioritaskan-b79ff4d77de8
)
